# Transfer Learning using Tensorflow 2

In [ ]:
# Downloadng and becoming one with the data
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

In [21]:
import os

dataset_path = "../datasets/10_food_classes_10_percent"
for dirpath, dirnames, filenames in os.walk(dataset_path):
    print(f"dirpath: { dirpath } { len(filenames) }")

dirpath: ../datasets/10_food_classes_10_percent 0
dirpath: ../datasets/10_food_classes_10_percent/test 0
dirpath: ../datasets/10_food_classes_10_percent/test/fried_rice 250
dirpath: ../datasets/10_food_classes_10_percent/test/grilled_salmon 250
dirpath: ../datasets/10_food_classes_10_percent/test/hamburger 250
dirpath: ../datasets/10_food_classes_10_percent/test/chicken_wings 250
dirpath: ../datasets/10_food_classes_10_percent/test/ramen 250
dirpath: ../datasets/10_food_classes_10_percent/test/sushi 250
dirpath: ../datasets/10_food_classes_10_percent/test/ice_cream 250
dirpath: ../datasets/10_food_classes_10_percent/test/chicken_curry 250
dirpath: ../datasets/10_food_classes_10_percent/test/steak 250
dirpath: ../datasets/10_food_classes_10_percent/test/pizza 250
dirpath: ../datasets/10_food_classes_10_percent/train 0
dirpath: ../datasets/10_food_classes_10_percent/train/fried_rice 75
dirpath: ../datasets/10_food_classes_10_percent/train/grilled_salmon 75
dirpath: ../datasets/10_food_cl

In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = dataset_path + '/train/'
test_dir = dataset_path + '/test/'

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

train_data_10_percent = train_datagen.flow_from_directory(directory=train_dir, target_size=IMAGE_SHAPE, batch_size=BATCH_SIZE, class_mode="categorical")
test_data_10_percent = test_datagen.flow_from_directory(directory=test_dir, target_size=IMAGE_SHAPE, batch_size=BATCH_SIZE, class_mode="categorical")


Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [23]:
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5"
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

### Setting up Callbacks

In [24]:
import datetime
import tensorflow as tf
import tensorflow_hub as hub
import tf_keras as keras


def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir)
    print(f"Saving TensorBoard log files to: { log_dir }")
    return tensorboard_callback

## Creating models using Tensorflow Hub

In [25]:

# m = tf.keras.Sequential([
#     tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False),
#     tf.keras.layers.Dense(5, activation='softmax')
# ])
# m.build([None, 224, 224, 3])  # Batch input shape.

In [26]:
def create_model(model_url, num_classes=10):
    feature_extraction_layer = hub.KerasLayer(model_url, 
                                             trainable=False, 
                                             name="feature_extraction_layer", 
                                             input_shape=IMAGE_SHAPE+(3, ))
    
    model = keras.Sequential([
        feature_extraction_layer,
        keras.layers.Dense(num_classes, activation="softmax", name="output_layer")
    ])
    return model

### Plot Loss Curves

In [27]:
import matplotlib.pyplot as plt

def plot_loss_curves(history):
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']

    plt.plot(loss, )
    plt.plot(val_loss, )
    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.legend()

    plt.figure()
    plt.plot(accuracy, )
    plt.plot(val_accuracy, )
    plt.title("Accuracy")
    plt.xlabel("Epochs")
    plt.legend()

### Resnet50

In [28]:
resnet_model = create_model(resnet_url, num_classes=train_data_10_percent.num_classes)

In [29]:
resnet_model.compile(loss="categorical_crossentropy",
                     optimizer=keras.optimizers.Adam(),
                     metrics=["accuracy"])

In [30]:
resnet_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extraction_layer (  (None, 2048)              23564800  
 KerasLayer)                                                     
                                                                 
 output_layer (Dense)        (None, 10)                20490     
                                                                 
Total params: 23585290 (89.97 MB)
Trainable params: 20490 (80.04 KB)
Non-trainable params: 23564800 (89.89 MB)
_________________________________________________________________


In [31]:
resnet_history = resnet_model.fit(train_data_10_percent,
                                  steps_per_epoch=len(train_data_10_percent),
                                  epochs=5,
                                  validation_data=test_data_10_percent)

Epoch 1/5
24/24 [==============================] - ETA: 0s - loss: 1.8554 - accuracy: 0.3880

KeyboardInterrupt: 

In [ ]:
plot_loss_curves(resnet_history)

### EfficientNet

In [ ]:
efficientnet_model = create_model(efficientnet_url, num_classes=train_data_10_percent.num_classes)
efficientnet_model.compile(loss="categorical_crossentropy",
                     optimizer=keras.optimizers.Adam(),
                     metrics=["accuracy"])
efficientnet_model.summary()

In [ ]:
efficientnet_history = efficientnet_model.fit(train_data_10_percent,
                                  steps_per_epoch=len(train_data_10_percent),
                                  epochs=5,
                                  validation_data=test_data_10_percent)

In [ ]:
plot_loss_curves(efficientnet_history)